### __Getting Json Data to Excel and Refresh Data on a Schedule__

#### Import Libraries

* __pandas:__ library for data manipulation and analysis
* __schedule:__ an in process scheduler for periodic jobs that uses the builder pattern for configuration. 
* __time:__ handle time-related tasks
* __xlsxwriter:__ for writing files in the Excel
* __openpyxl:__ to read/write Excel xlsx/xlsm/xltx/xltm files
* __datetime:__ encapsulation of date/time values
* __Workbook:__ there is no need to create a file on the filesystem to get started with openpyxl if you import workbook
* __load_workbook:__ loading from a file

In [3]:
import pandas as pd
import schedule
import time
import xlsxwriter 
import openpyxl
import datetime
from openpyxl import Workbook
from openpyxl import load_workbook

#### Data

In this Json file, exchange and gold rates are given in Turkish Lira. This file is updated every 15 minutes.
<br>Convert the Json page to dataframe with pandas module 'read_json' and dataframe is ready. 
<br>There are 3 rows and many columns.
<br>First row is 'Alış' is mean 'Buying'
<br>Second row is 'Satış' is mean 'Sales'
<br>Last row is 'Tür' is mean 'Kind' (Döviz(Exchange), Altın(Gold))
<br>First column is 'Güncelleme Tarihi' is mean 'Time of update' 
<br>Other columns are names of exchange and gold types. 

In [4]:
data_df = pd.read_json('https://finans.truncgil.com/today.json')

In [13]:
data_df = data_df.reindex(["Tür", "Alış", "Satış"])

In [14]:
data_df

,Güncelleme Tarihi,ABD DOLARI,AVUSTRALYA DOLARI,DANİMARKA KRONU,EURO,İNGİLİZ STERLİNİ,İSVİÇRE FRANGI,İSVEÇ KRONU,KANADA DOLARI,KUVEYT DİNARI,...,Ata Altın,Reşat Altın,Hamit Altın,İkibuçuk Altın,Gremse Altın,Beşli Altın,14 Ayar Altın,18 Ayar Altın,22 Ayar Bilezik,Gümüş
Tür,2020-07-03 09:30:02,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,...,Altın,Altın,Altın,Altın,Altın,Altın,Altın,Altın,Altın,Altın
Alış,2020-07-03 09:30:02,6.8428,4.7296,1.0346,7.7211,8.5454,7.2330,0.73559,5.0281,22.1111,...,"2.589,35","2.583,63","2.583,63","6.294,01","6.294,01","12.588,02","228,34","293,59","358,81","3,97"
Satış,2020-07-03 09:30:02,6.8552,4.7604,1.0397,7.7350,8.5900,7.2794,0.74320,5.0508,22.4005,...,"2.615,50","2.609,73","2.609,73","6.357,58","6.357,58","12.954,31","230,64","296,56","362,44","3,97"


We have read the data, defined it as a dataframe and now it's time to export it as Excel.

In [6]:
data_df.to_excel("sch.xlsx", header=True)

We want to update the data every 15 minutes after exporting the first data. 
<br>In order to repeat this process, we first define the job.
<br>In job;
<br> we load the workbook in which we export the first data.
<br> we read the json data again and we define it again without last row.
<br>then we export it without header on the last line of the Excel we loaded. 
<br> use datetime to see when the job is running.

In [7]:
def job():
    book = load_workbook('sch.xlsx')
    writer = pd.ExcelWriter('sch.xlsx', engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}
    data_df1 = pd.read_json('https://finans.truncgil.com/today.json')
    data_df1 = data_df1.drop('Tür', axis=0)
    for sheetname in writer.sheets:
        data_df1.to_excel(writer, sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, header= False)
    print('Last Run:', datetime.datetime.now())    
    writer.save() 


and we scheduled to do this job every 15 minutes.

In [8]:
schedule.every(15).minutes.do(job)

Every 15 minutes do job() (last run: [never], next run: 2020-07-03 12:56:41)

In [9]:
while True:
    schedule.run_pending()
    time.sleep(1)

Last Run: 2020-07-03 12:56:41.437040
Last Run: 2020-07-03 13:11:42.120179
Last Run: 2020-07-03 13:26:43.215500


KeyboardInterrupt: 

Let's look at the last version of our data after the job stops. 

In [12]:
lastdf = pd.read_excel('sch.xlsx')
lastdf

,Unnamed: 0,Güncelleme Tarihi,ABD DOLARI,AVUSTRALYA DOLARI,DANİMARKA KRONU,EURO,İNGİLİZ STERLİNİ,İSVİÇRE FRANGI,İSVEÇ KRONU,KANADA DOLARI,...,Ata Altın,Reşat Altın,Hamit Altın,İkibuçuk Altın,Gremse Altın,Beşli Altın,14 Ayar Altın,18 Ayar Altın,22 Ayar Bilezik,Gümüş
0,Alış,2020-07-03 09:30:02,6.8428,4.7296,1.0346,7.7211,8.5454,7.2330,0.73559,5.0281,...,"2.589,35","2.583,63","2.583,63","6.294,01","6.294,01","12.588,02","228,34","293,59","358,81","3,97"
1,Satış,2020-07-03 09:30:02,6.8552,4.7604,1.0397,7.7350,8.5900,7.2794,0.74320,5.0508,...,"2.615,50","2.609,73","2.609,73","6.357,58","6.357,58","12.954,31","230,64","296,56","362,44","3,97"
2,Tür,2020-07-03 09:30:02,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,Döviz,...,Altın,Altın,Altın,Altın,Altın,Altın,Altın,Altın,Altın,Altın
3,Alış,2020-07-03 09:45:02,6.8428,4.7296,1.0346,7.7211,8.5454,7.2330,0.73559,5.0281,...,"2.588,03","2.582,32","2.582,32","6.290,81","6.290,81","12.581,61","228,22","293,45","358,63","3,97"
4,Satış,2020-07-03 09:45:02,6.8552,4.7604,1.0397,7.7350,8.5900,7.2794,0.74320,5.0508,...,"2.614,17","2.608,40","2.608,40","6.354,35","6.354,35","12.947,72","230,53","296,41","362,25","3,97"
5,Alış,2020-07-03 10:00:02,6.8428,4.7296,1.0346,7.7211,8.5454,7.2330,0.73559,5.0281,...,"2.588,42","2.582,70","2.582,70","6.291,74","6.291,74","12.583,48","228,21","293,42","358,60","3,97"
6,Satış,2020-07-03 10:00:02,6.8552,4.7604,1.0397,7.7350,8.5900,7.2794,0.74320,5.0508,...,"2.614,56","2.608,79","2.608,79","6.355,29","6.355,29","12.949,64","230,51","296,39","362,23","3,97"
7,Alış,2020-07-03 10:15:02,6.8428,4.7296,1.0346,7.7211,8.5454,7.2330,0.73559,5.0281,...,"2.588,26","2.582,55","2.582,55","6.291,37","6.291,37","12.582,73","228,24","293,47","358,66","3,97"
8,Satış,2020-07-03 10:15:02,6.8552,4.7604,1.0397,7.7350,8.5900,7.2794,0.74320,5.0508,...,"2.614,41","2.608,64","2.608,64","6.354,92","6.354,92","12.948,88","230,55","296,44","362,28","3,97"
